In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np
import os

import torch
import torch.nn.functional as F
import torch.optim as optim
import networkx as nx
from scipy import sparse
from scipy.linalg import fractional_matrix_power

from utils import *
from models import GNN
from dataset_utils import DataLoader

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.019,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-2,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=128,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.9,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--dataset', default='citeseer', help='Dataset name.')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='citeseer', type=None, choices=None, help='Dataset name.', metavar=None)

In [3]:
args = parser.parse_args("")

In [4]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)

In [5]:
dname = args.dataset
dataset = DataLoader(dname)
data = dataset[0]

In [6]:
A_norm, A, X, labels, idx_train, idx_val, idx_test = load_citation_data(data)

In [7]:
G = nx.from_numpy_matrix(A)
feature_dictionary = {}

for i in np.arange(len(labels)):
    feature_dictionary[i] = labels[i]

nx.set_node_attributes(G, feature_dictionary, "attr_name")

In [8]:
sub_graphs = []

for i in np.arange(len(A)):
    s_indexes = []
    for j in np.arange(len(A)):
        s_indexes.append(i)
        if(A[i][j]==1):
            s_indexes.append(j)
    sub_graphs.append(G.subgraph(s_indexes))

subgraph_nodes_list = []

for i in np.arange(len(sub_graphs)):
    subgraph_nodes_list.append(list(sub_graphs[i].nodes))

In [9]:
sub_graphs_adj = []
for index in np.arange(len(sub_graphs)):
    sub_graphs_adj.append(nx.adjacency_matrix(sub_graphs[index]).toarray())

In [10]:
new_adj = torch.zeros(A.shape[0], A.shape[0])

for node in np.arange(len(subgraph_nodes_list)):
    sub_adj = sub_graphs_adj[node]
    for neighbors in np.arange(len(subgraph_nodes_list[node])):
        index = subgraph_nodes_list[node][neighbors]
        count = torch.tensor(0).float()
        if(index==node):
            continue
        else:
            c_neighbors = set(subgraph_nodes_list[node]).intersection(subgraph_nodes_list[index])
            if index in c_neighbors:
                nodes_list = subgraph_nodes_list[node]
                sub_graph_index = nodes_list.index(index)
                c_neighbors_list = list(c_neighbors)
                for i, item1 in enumerate(nodes_list):
                    if(item1 in c_neighbors):
                        for item2 in c_neighbors_list:
                            j = nodes_list.index(item2)
                            count += sub_adj[i][j]

            new_adj[node][index] = count/2
            new_adj[node][index] = new_adj[node][index]/(len(c_neighbors)*(len(c_neighbors)-1))
            new_adj[node][index] = new_adj[node][index] * (len(c_neighbors)**1)

In [11]:
features = torch.FloatTensor(X)
labels = torch.LongTensor(labels)

weight = torch.FloatTensor(new_adj)
weight = weight / weight.sum(1, keepdim=True)

weight = weight + torch.FloatTensor(A)

coeff = weight.sum(1, keepdim=True)
coeff = torch.diag((coeff.T)[0])

weight = weight + coeff

In [12]:
weight = weight.detach().numpy()
weight = np.nan_to_num(weight, nan=0)

row_sum = np.array(np.sum(weight, axis=1))
degree_matrix = np.matrix(np.diag(row_sum+1))

D = fractional_matrix_power(degree_matrix, -0.5)
A_tilde_hat = D.dot(weight).dot(D)

In [13]:
adj = torch.FloatTensor(A_tilde_hat)

In [14]:
# Model and optimizer
model = GNN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

In [15]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately, deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [16]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [ ]:
# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 1.8565 acc_train: 0.1833 loss_val: 1.7710 acc_val: 0.1880 time: 0.7142s
Epoch: 0002 loss_train: 1.7756 acc_train: 0.2167 loss_val: 1.7379 acc_val: 0.3060 time: 0.7780s
Epoch: 0003 loss_train: 1.6989 acc_train: 0.3500 loss_val: 1.7081 acc_val: 0.4180 time: 0.7617s
Epoch: 0004 loss_train: 1.6287 acc_train: 0.3917 loss_val: 1.6795 acc_val: 0.4700 time: 0.8227s
Epoch: 0005 loss_train: 1.6024 acc_train: 0.4583 loss_val: 1.6568 acc_val: 0.5120 time: 0.7600s
Epoch: 0006 loss_train: 1.5413 acc_train: 0.5583 loss_val: 1.6378 acc_val: 0.5220 time: 0.9026s
Epoch: 0007 loss_train: 1.4961 acc_train: 0.6333 loss_val: 1.6243 acc_val: 0.5640 time: 0.9345s
Epoch: 0008 loss_train: 1.5057 acc_train: 0.5917 loss_val: 1.6191 acc_val: 0.5820 time: 1.1795s
Epoch: 0009 loss_train: 1.4694 acc_train: 0.7250 loss_val: 1.6178 acc_val: 0.5940 time: 0.8417s
Epoch: 0010 loss_train: 1.4753 acc_train: 0.6917 loss_val: 1.6203 acc_val: 0.6340 time: 0.7739s
Epoch: 0011 loss_train: 1.4720 acc_train

Epoch: 0087 loss_train: 0.7304 acc_train: 0.8083 loss_val: 1.1657 acc_val: 0.7060 time: 0.9065s
Epoch: 0088 loss_train: 0.8136 acc_train: 0.7750 loss_val: 1.1582 acc_val: 0.7080 time: 0.9199s
Epoch: 0089 loss_train: 0.7722 acc_train: 0.7500 loss_val: 1.1687 acc_val: 0.7000 time: 0.9219s
Epoch: 0090 loss_train: 0.7648 acc_train: 0.7667 loss_val: 1.1954 acc_val: 0.6780 time: 0.9270s
Epoch: 0091 loss_train: 0.7304 acc_train: 0.8000 loss_val: 1.2118 acc_val: 0.6700 time: 0.9552s
Epoch: 0092 loss_train: 0.7279 acc_train: 0.8167 loss_val: 1.2058 acc_val: 0.6800 time: 0.9123s
Epoch: 0093 loss_train: 0.8071 acc_train: 0.8250 loss_val: 1.1955 acc_val: 0.6800 time: 0.9453s
Epoch: 0094 loss_train: 0.7634 acc_train: 0.8000 loss_val: 1.1927 acc_val: 0.6760 time: 0.9180s
Epoch: 0095 loss_train: 0.8028 acc_train: 0.7667 loss_val: 1.1867 acc_val: 0.6920 time: 0.9261s
Epoch: 0096 loss_train: 0.7581 acc_train: 0.8250 loss_val: 1.1625 acc_val: 0.6920 time: 0.9391s
Epoch: 0097 loss_train: 0.7597 acc_train

Epoch: 0173 loss_train: 0.7399 acc_train: 0.8167 loss_val: 1.1606 acc_val: 0.6780 time: 0.7689s
Epoch: 0174 loss_train: 0.7113 acc_train: 0.8417 loss_val: 1.1490 acc_val: 0.6880 time: 0.7846s
Epoch: 0175 loss_train: 0.7066 acc_train: 0.8417 loss_val: 1.1405 acc_val: 0.7100 time: 0.7670s
Epoch: 0176 loss_train: 0.7250 acc_train: 0.7917 loss_val: 1.1329 acc_val: 0.7060 time: 0.8029s
Epoch: 0177 loss_train: 0.5777 acc_train: 0.8667 loss_val: 1.1324 acc_val: 0.7140 time: 0.7889s
Epoch: 0178 loss_train: 0.6608 acc_train: 0.8333 loss_val: 1.1390 acc_val: 0.7120 time: 0.7915s
Epoch: 0179 loss_train: 0.5956 acc_train: 0.8750 loss_val: 1.1537 acc_val: 0.6920 time: 0.7689s
Epoch: 0180 loss_train: 0.6169 acc_train: 0.8333 loss_val: 1.1738 acc_val: 0.6760 time: 0.7969s
Epoch: 0181 loss_train: 0.6511 acc_train: 0.8667 loss_val: 1.1737 acc_val: 0.6660 time: 0.7440s
Epoch: 0182 loss_train: 0.5893 acc_train: 0.8333 loss_val: 1.1609 acc_val: 0.6760 time: 0.7899s
Epoch: 0183 loss_train: 0.7206 acc_train